In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [2]:
class Argument():
    def __init__(self, batch_size=128, test_batch_size=1000, epochs=14, lr=1.0,
                gamma=0.7,no_cuda=False, save_model=False):
        
        self.batch_size = batch_size
        self.test_batch_size = test_batch_size
        self.epochs = epochs
        self.lr = lr
        self.gamma = gamma
        self.no_cuda = no_cuda
        self.save_model = save_model


In [3]:
args = Argument()
use_cuda = not args.no_cuda and torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

In [4]:
#%%time

#import data_saver
#data_saver.save_data()

Wall time: 11.2 s


In [5]:
%%time

train_data = torch.load('data/train_data.pt', map_location=device)
train_labels = torch.load('data/train_labels.pt', map_location=device)

test_data = torch.load('data/test_data.pt', map_location=device)
test_labels = torch.load('data/test_labels.pt', map_location=device)

Wall time: 1.9 s


In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(1600, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

def train(args, model, device, optimizer):
    batch_size = args.batch_size
    model.train()
    for i in range(len(train_data)//batch_size):
        data = train_data[batch_size*i:batch_size*(i+1)]
        target = train_labels[batch_size*i:batch_size*(i+1)]
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


def test(args, model, device, epoch):
    model.eval()
    batch_size = args.test_batch_size
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i in range(len(test_data)//batch_size):
            data = test_data[batch_size*i:batch_size*(i+1)]
            target = test_labels[batch_size*i:batch_size*(i+1)]
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_data)

    print('\n Epoch {} Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        epoch, test_loss, correct, len(test_data),
        100. * correct / len(test_data)))
    return 100.*correct / len(test_data)


def main():
    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, optimizer)
        test_acc = test(args, model, device, epoch)
        if test_acc >= 99:
            return
        scheduler.step()

    if args.save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")


In [7]:
%%timeit
main()


 Epoch 1 Test set: Average loss: 0.0422, Accuracy: 9859/10000 (99%)


 Epoch 2 Test set: Average loss: 0.0331, Accuracy: 9888/10000 (99%)


 Epoch 3 Test set: Average loss: 0.0250, Accuracy: 9912/10000 (99%)


 Epoch 1 Test set: Average loss: 0.0477, Accuracy: 9843/10000 (98%)


 Epoch 2 Test set: Average loss: 0.0294, Accuracy: 9895/10000 (99%)


 Epoch 3 Test set: Average loss: 0.0254, Accuracy: 9915/10000 (99%)


 Epoch 1 Test set: Average loss: 0.0424, Accuracy: 9863/10000 (99%)


 Epoch 2 Test set: Average loss: 0.0298, Accuracy: 9893/10000 (99%)


 Epoch 3 Test set: Average loss: 0.0269, Accuracy: 9908/10000 (99%)


 Epoch 1 Test set: Average loss: 0.0426, Accuracy: 9850/10000 (98%)


 Epoch 2 Test set: Average loss: 0.0360, Accuracy: 9886/10000 (99%)


 Epoch 3 Test set: Average loss: 0.0270, Accuracy: 9907/10000 (99%)


 Epoch 1 Test set: Average loss: 0.0531, Accuracy: 9819/10000 (98%)


 Epoch 2 Test set: Average loss: 0.0310, Accuracy: 9884/10000 (99%)


 Epoch 3 Test set: 